In [11]:
import pandas as pd
import re
import numpy as np

In [12]:
df = pd.read_csv('flipkart (1).csv')

# Display original column names
print("Original columns:", df.columns.tolist())
print(f"Original shape: {df.shape}")

# Step 1: Assign proper column names based on the data structure
column_names = [
    'product_name',
    'rating',
    'num_ratings',
    'and_symbol_1',
    'num_reviews',
    'screen_type',
    'model_id',
    'launch_year',
    'sound_output',
    'warranty',
    'current_price',
    'original_price',
    'discount_percent',
    'exchange_label',
    'exchange_value'
]

df.columns = column_names

# Step 2: Remove completely empty rows
df = df.dropna(how='all')

# Step 3: Remove rows where product_name is empty or very short
df = df[df['product_name'].notna()]
df = df[df['product_name'].str.len() > 10]


Original columns: ['KzDlHZ', 'XQDdHH', 'Wphh3N', 'hG7V+4', 'Wphh3N 2', 'J+igdf', 'J+igdf 2', 'J+igdf 3', 'J+igdf 4', 'J+igdf 5', 'Nx9bqj', 'yRaY8j', 'UkUFwK', 'yiggsN 2', 'yiggsN 3']
Original shape: (1126, 15)


In [13]:
def clean_price(price_str):
    """Extract numeric price from string like '₹18,889'"""
    if pd.isna(price_str):
        return np.nan
    # Remove currency symbols and commas
    cleaned = re.sub(r'[₹,]', '', str(price_str))
    try:
        return float(cleaned)
    except:
        return np.nan

def clean_rating(rating_str):
    """Extract rating number"""
    if pd.isna(rating_str):
        return np.nan
    try:
        return float(rating_str)
    except:
        return np.nan

def clean_number(num_str):
    """Extract numbers from strings like '34,911 Ratings'"""
    if pd.isna(num_str):
        return np.nan
    # Extract numbers and remove commas
    match = re.search(r'[\d,]+', str(num_str))
    if match:
        cleaned = re.sub(r',', '', match.group())
        try:
            return int(cleaned)
        except:
            return np.nan
    return np.nan

def clean_discount(discount_str):
    """Extract discount percentage"""
    if pd.isna(discount_str):
        return np.nan
    # Extract number before '%'
    match = re.search(r'(\d+)%', str(discount_str))
    if match:
        return int(match.group(1))
    return np.nan

def extract_screen_info(screen_str):
    """Extract screen resolution and type"""
    if pd.isna(screen_str):
        return np.nan, np.nan
    parts = str(screen_str).split('|')
    resolution = parts[0].strip() if len(parts) > 0 else np.nan
    display_type = parts[1].strip() if len(parts) > 1 else np.nan
    return resolution, display_type

def extract_year(year_str):
    """Extract year from 'Launch Year: 2025'"""
    if pd.isna(year_str):
        return np.nan
    match = re.search(r'(\d{4})', str(year_str))
    if match:
        return int(match.group(1))
    return np.nan

def extract_watts(sound_str):
    """Extract watts from 'Total Sound Output: 50 W'"""
    if pd.isna(sound_str):
        return np.nan
    match = re.search(r'(\d+)\s*W', str(sound_str))
    if match:
        return int(match.group(1))
    return np.nan

def extract_model_id(model_str):
    """Clean model ID"""
    if pd.isna(model_str):
        return np.nan
    # Extract everything after 'Model ID:'
    match = re.search(r'Model ID:\s*(.+)', str(model_str))
    if match:
        return match.group(1).strip()
    return str(model_str).strip()

def clean_warranty(warranty_str):
    """Clean warranty text"""
    if pd.isna(warranty_str):
        return np.nan
    # Remove extra spaces and special characters
    cleaned = re.sub(r'\s+', ' ', str(warranty_str))
    return cleaned.strip()

# Apply cleaning functions
print("\nCleaning data...")

df['rating'] = df['rating'].apply(clean_rating)
df['num_ratings'] = df['num_ratings'].apply(clean_number)
df['num_reviews'] = df['num_reviews'].apply(clean_number)
df['current_price'] = df['current_price'].apply(clean_price)
df['original_price'] = df['original_price'].apply(clean_price)
df['discount_percent'] = df['discount_percent'].apply(clean_discount)
df['exchange_value'] = df['exchange_value'].apply(clean_price)

# Extract screen information
df[['screen_resolution', 'display_type']] = df['screen_type'].apply(
    lambda x: pd.Series(extract_screen_info(x))
)

df['launch_year'] = df['launch_year'].apply(extract_year)
df['sound_watts'] = df['sound_output'].apply(extract_watts)
df['model_id_clean'] = df['model_id'].apply(extract_model_id)
df['warranty_clean'] = df['warranty'].apply(clean_warranty)

# Extract brand from product name
def extract_brand(product_name):
    """Extract brand name (first word typically)"""
    if pd.isna(product_name):
        return np.nan
    words = str(product_name).split()
    return words[0] if words else np.nan

df['brand'] = df['product_name'].apply(extract_brand)

# Extract screen size from product name
def extract_screen_size(product_name):
    """Extract screen size in cm and inches"""
    if pd.isna(product_name):
        return np.nan, np.nan
    # Pattern: "108 cm (43 inch)"
    pattern = r'(\d+)\s*cm\s*\((\d+)\s*inch\)'
    match = re.search(pattern, str(product_name))
    if match:
        return int(match.group(1)), int(match.group(2))
    return np.nan, np.nan

df[['screen_size_cm', 'screen_size_inch']] = df['product_name'].apply(
    lambda x: pd.Series(extract_screen_size(x))
)

# Calculate discount amount
df['discount_amount'] = df['original_price'] - df['current_price']



Cleaning data...


In [14]:
final_columns = [
    'brand',
    'product_name',
    'screen_size_cm',
    'screen_size_inch',
    'screen_resolution',
    'display_type',
    'rating',
    'num_ratings',
    'num_reviews',
    'current_price',
    'original_price',
    'discount_percent',
    'discount_amount',
    'exchange_value',
    'launch_year',
    'sound_watts',
    'model_id_clean',
    'warranty_clean'
]

df_clean = df[final_columns].copy()

# Step 6: Remove rows with critical missing data
df_clean = df_clean[df_clean['current_price'].notna()]
df_clean = df_clean[df_clean['brand'].notna()]

# Reset index
df_clean = df_clean.reset_index(drop=True)

# Step 7: Display summary
print(f"\nCleaned shape: {df_clean.shape}")
print(f"\nRemoved {len(df) - len(df_clean)} rows with missing critical data")
print("\nFirst few rows:")
print(df_clean.head())
print("\nData types:")
print(df_clean.dtypes)
print("\nMissing values:")
print(df_clean.isnull().sum())
print("\nBasic statistics:")
print(df_clean.describe())

# Save cleaned data
df_clean.to_csv('flipkart_cleaned.csv', index=False)
print("\n✓ Cleaned data saved to 'flipkart_cleaned.csv'")

# Additional insights
print("\n" + "="*50)
print("DATA SUMMARY")
print("="*50)
print(f"Total products: {len(df_clean)}")
print(f"Unique brands: {df_clean['brand'].nunique()}")
print(f"Top 5 brands:")
print(df_clean['brand'].value_counts().head())
print(f"\nPrice range: ₹{df_clean['current_price'].min():,.0f} - ₹{df_clean['current_price'].max():,.0f}")
print(f"Average price: ₹{df_clean['current_price'].mean():,.0f}")
print(f"Average discount: {df_clean['discount_percent'].mean():.1f}%")
print(f"Average rating: {df_clean['rating'].mean():.2f}")



Cleaned shape: (1080, 18)

Removed 0 rows with missing critical data

First few rows:
       brand                                       product_name  \
0    Thomson  Thomson Phoenix 2025 Edition 108 cm (43 inch) ...   
1    Thomson  Thomson Phoenix 2025 Edition 126 cm (50 inch) ...   
2     Foxsky  Foxsky 80 cm (32 inch) HD Ready LED Smart Andr...   
3  Blaupunkt  Blaupunkt Quantum Dot 100 cm (40 inch) QLED Fu...   
4       SONY  SONY BRAVIA 2 II 138.8 cm (55 inch) Ultra HD (...   

   screen_size_cm  screen_size_inch screen_resolution display_type  rating  \
0           108.0              43.0     Ultra HD (4K)         QLED     4.3   
1           126.0              50.0     Ultra HD (4K)         QLED     4.3   
2            80.0              32.0          HD Ready          LED     4.1   
3           100.0              40.0           Full HD         QLED     4.4   
4             8.0              55.0     Ultra HD (4K)          LED     4.7   

   num_ratings  num_reviews  current_pric

In [15]:
df_clean.head(100)

,brand,product_name,screen_size_cm,screen_size_inch,screen_resolution,display_type,rating,num_ratings,num_reviews,current_price,original_price,discount_percent,discount_amount,exchange_value,launch_year,sound_watts,model_id_clean,warranty_clean
0,Thomson,Thomson Phoenix 2025 Edition 108 cm (43 inch) ...,108.0,43.0,Ultra HD (4K),QLED,4.3,34911.0,5706.0,18889.0,28999.0,34.0,10110.0,5400.0,2025,50,43QAI1005,1 Year Warranty on Product
1,Thomson,Thomson Phoenix 2025 Edition 126 cm (50 inch) ...,126.0,50.0,Ultra HD (4K),QLED,4.3,34911.0,5706.0,23999.0,40999.0,41.0,17000.0,5400.0,2025,48,50QAI1015,1 Year Warranty on Product
2,Foxsky,Foxsky 80 cm (32 inch) HD Ready LED Smart Andr...,80.0,32.0,HD Ready,LED,4.1,12502.0,2521.0,6999.0,22499.0,68.0,15500.0,NaN,2023,30,32 FSELS PRO,1-Year Foxsky Standard Warranty on Product fro...
3,Blaupunkt,Blaupunkt Quantum Dot 100 cm (40 inch) QLED Fu...,100.0,40.0,Full HD,QLED,4.4,12512.0,2934.0,13799.0,19999.0,31.0,6200.0,5400.0,2025,48,40QD7070,1 Year Warranty on Product & 6 Months on Acces...
4,SONY,SONY BRAVIA 2 II 138.8 cm (55 inch) Ultra HD (...,8.0,55.0,Ultra HD (4K),LED,4.7,27797.0,3527.0,55990.0,91900.0,39.0,35910.0,5400.0,2024,20,K-55S25EM2,1 year warranty provided by the manufacturer f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LG,LG AI TV UA8200 126 cm (50 inch) Ultra HD (4K)...,126.0,50.0,Ultra HD (4K),LED,4.4,31838.0,2178.0,35990.0,64490.0,44.0,28500.0,NaN,2025,20,50UA82006LA,Warranty Information: 1-Year LG India Standard...
96,TCL,TCL T6C 139 cm (55 inch) QLED Ultra HD (4K) Sm...,139.0,55.0,Ultra HD (4K),QLED,4.0,2487.0,175.0,32990.0,93990.0,64.0,61000.0,5400.0,2025,24,55T6C,2 Years Product Warranty
97,Samsung,Samsung Crystal 4K Infinity Vision 125 cm (50 ...,125.0,50.0,Ultra HD (4K),LED,4.3,64260.0,4359.0,36990.0,52400.0,29.0,15410.0,5400.0,2025,20,UA50UE84AFULXL,2 Year Warranty (1 Year Standard Warranty + 1 ...
98,TCL,TCL C6K 139 cm (55 inch) Ultra HD (4K) Mini LE...,139.0,55.0,Ultra HD (4K),Mini LED,4.1,585.0,46.0,45990.0,119990.0,61.0,74000.0,7900.0,2025,40,55C6K,2 Years Product Warranty


In [16]:
df_clean['display_type'].unique()

array(['QLED', 'LED', 'Mini LED', 'OLED', 'QNED', 'Neo QLED', 'NanoCell'],
      dtype=object)

In [17]:
df_clean['brand'].unique()

array(['Thomson', 'Foxsky', 'Blaupunkt', 'SONY', 'Coocaa', 'XIAOMI', 'LG',
       'BESTON', 'MarQ', 'acerpure', 'Infinix', 'Uniboom', 'Samsung',
       'TCL', 'realme', 'iFFALCON', 'Reliance', 'KODAK', 'MOTOROLA',
       'PHILIPS', 'Hisense', 'InnoQ', 'Acer', 'Lloyd', 'Panasonic',
       'TOSHIBA', 'Vu', 'Daiwa', 'Haier', 'ONIDA', 'HUIDI', 'Sansui',
       'Mi', 'Kenstar', 'Micromax', 'Compaq', 'REDMI', 'Admiral', 'Metz',
       'Nokia', 'BPL', 'Aiwa', 'OnePlus', 'IMPEX', 'Samtonic', 'Hyundai',
       'NVY', 'VZY', 'VOIR', 'XElectron', 'LIMEBERRY', 'ZEBRONICS',
       'Reintech', 'Weston', 'Doodle', 'skylive', 'Elista', 'iMEE',
       'Qthin', 'IAIR', 'INVANTER', 'LEEMA', 'NU', 'JVC', 'LUGOSI',
       'Onix', 'Dish', 'VONE', 'EARTHONIC', 'Asiano', 'cenit', 'Swan',
       'Dor'], dtype=object)

In [18]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brand              1080 non-null   object 
 1   product_name       1080 non-null   object 
 2   screen_size_cm     1071 non-null   float64
 3   screen_size_inch   1071 non-null   float64
 4   screen_resolution  1080 non-null   object 
 5   display_type       1080 non-null   object 
 6   rating             846 non-null    float64
 7   num_ratings        846 non-null    float64
 8   num_reviews        846 non-null    float64
 9   current_price      1080 non-null   float64
 10  original_price     1070 non-null   float64
 11  discount_percent   1070 non-null   float64
 12  discount_amount    1070 non-null   float64
 13  exchange_value     515 non-null    float64
 14  launch_year        1080 non-null   int64  
 15  sound_watts        1080 non-null   int64  
 16  model_id_clean     1080 

In [19]:
df_clean.isnull().sum()

brand                  0
product_name           0
screen_size_cm         9
screen_size_inch       9
screen_resolution      0
display_type           0
rating               234
num_ratings          234
num_reviews          234
current_price          0
original_price        10
discount_percent      10
discount_amount       10
exchange_value       565
launch_year            0
sound_watts            0
model_id_clean         0
warranty_clean        25
dtype: int64

In [20]:
df = pd.read_csv('flipkart (1).csv')

# Display original column names
print("Original columns:", df.columns.tolist())
print(f"Original shape: {df.shape}")

# Step 1: Assign proper column names based on the data structure
column_names = [
    'product_name',
    'rating',
    'num_ratings',
    'and_symbol_1',
    'num_reviews',
    'screen_type',
    'model_id',
    'launch_year',
    'sound_output',
    'warranty',
    'current_price',
    'original_price',
    'discount_percent',
    'exchange_label',
    'exchange_value'
]

df.columns = column_names

# Step 2: Remove completely empty rows
df = df.dropna(how='all')

# Step 3: Remove rows where product_name is empty or very short
df = df[df['product_name'].notna()]
df = df[df['product_name'].str.len() > 10]


Original columns: ['KzDlHZ', 'XQDdHH', 'Wphh3N', 'hG7V+4', 'Wphh3N 2', 'J+igdf', 'J+igdf 2', 'J+igdf 3', 'J+igdf 4', 'J+igdf 5', 'Nx9bqj', 'yRaY8j', 'UkUFwK', 'yiggsN 2', 'yiggsN 3']
Original shape: (1126, 15)


In [21]:
def clean_price(price_str):
    """Extract numeric price from string like '₹18,889'"""
    if pd.isna(price_str):
        return np.nan
    # Remove currency symbols and commas
    cleaned = re.sub(r'[₹,]', '', str(price_str))
    try:
        return float(cleaned)
    except:
        return np.nan

def clean_rating(rating_str):
    """Extract rating number"""
    if pd.isna(rating_str):
        return np.nan
    try:
        return float(rating_str)
    except:
        return np.nan

def clean_number(num_str):
    """Extract numbers from strings like '34,911 Ratings'"""
    if pd.isna(num_str):
        return np.nan
    # Extract numbers and remove commas
    match = re.search(r'[\d,]+', str(num_str))
    if match:
        cleaned = re.sub(r',', '', match.group())
        try:
            return int(cleaned)
        except:
            return np.nan
    return np.nan

def clean_discount(discount_str):
    """Extract discount percentage"""
    if pd.isna(discount_str):
        return np.nan
    # Extract number before '%'
    match = re.search(r'(\d+)%', str(discount_str))
    if match:
        return int(match.group(1))
    return np.nan

def extract_screen_info(screen_str):
    """Extract screen resolution and type"""
    if pd.isna(screen_str):
        return np.nan, np.nan
    parts = str(screen_str).split('|')
    resolution = parts[0].strip() if len(parts) > 0 else np.nan
    display_type = parts[1].strip() if len(parts) > 1 else np.nan
    return resolution, display_type

def extract_year(year_str):
    """Extract year from 'Launch Year: 2025'"""
    if pd.isna(year_str):
        return np.nan
    match = re.search(r'(\d{4})', str(year_str))
    if match:
        return int(match.group(1))
    return np.nan

def extract_watts(sound_str):
    """Extract watts from 'Total Sound Output: 50 W'"""
    if pd.isna(sound_str):
        return np.nan
    match = re.search(r'(\d+)\s*W', str(sound_str))
    if match:
        return int(match.group(1))
    return np.nan

def extract_model_id(model_str):
    """Clean model ID"""
    if pd.isna(model_str):
        return np.nan
    # Extract everything after 'Model ID:'
    match = re.search(r'Model ID:\s*(.+)', str(model_str))
    if match:
        return match.group(1).strip()
    return str(model_str).strip()

def clean_warranty(warranty_str):
    """Clean warranty text"""
    if pd.isna(warranty_str):
        return np.nan
    # Remove extra spaces and special characters
    cleaned = re.sub(r'\s+', ' ', str(warranty_str))
    return cleaned.strip()

# Apply cleaning functions
print("\nCleaning data...")

df['rating'] = df['rating'].apply(clean_rating)
df['num_ratings'] = df['num_ratings'].apply(clean_number)
df['num_reviews'] = df['num_reviews'].apply(clean_number)
df['current_price'] = df['current_price'].apply(clean_price)
df['original_price'] = df['original_price'].apply(clean_price)
df['discount_percent'] = df['discount_percent'].apply(clean_discount)
df['exchange_value'] = df['exchange_value'].apply(clean_price)

# Extract screen information
df[['screen_resolution', 'display_type']] = df['screen_type'].apply(
    lambda x: pd.Series(extract_screen_info(x))
)

df['launch_year'] = df['launch_year'].apply(extract_year)
df['sound_watts'] = df['sound_output'].apply(extract_watts)
df['model_id_clean'] = df['model_id'].apply(extract_model_id)
df['warranty_clean'] = df['warranty'].apply(clean_warranty)

# Extract brand from product name
def extract_brand(product_name):
    """Extract brand name (first word typically)"""
    if pd.isna(product_name):
        return np.nan
    words = str(product_name).split()
    return words[0] if words else np.nan

df['brand'] = df['product_name'].apply(extract_brand)

# Extract screen size from product name
def extract_screen_size(product_name):
    """Extract screen size in cm and inches"""
    if pd.isna(product_name):
        return np.nan, np.nan
    # Pattern: "108 cm (43 inch)"
    pattern = r'(\d+)\s*cm\s*\((\d+)\s*inch\)'
    match = re.search(pattern, str(product_name))
    if match:
        return int(match.group(1)), int(match.group(2))
    return np.nan, np.nan

df[['screen_size_cm', 'screen_size_inch']] = df['product_name'].apply(
    lambda x: pd.Series(extract_screen_size(x))
)

# Calculate discount amount
df['discount_amount'] = df['original_price'] - df['current_price']



Cleaning data...


In [22]:
final_columns = [
    'brand',
    'product_name',
    'screen_size_cm',
    'screen_size_inch',
    'screen_resolution',
    'display_type',
    'rating',
    'num_ratings',
    'num_reviews',
    'current_price',
    'original_price',
    'discount_percent',
    'discount_amount',
    'exchange_value',
    'launch_year',
    'sound_watts',
    'model_id_clean',
    'warranty_clean'
]

df_clean = df[final_columns].copy()

# Step 6: Remove rows with critical missing data
df_clean = df_clean[df_clean['current_price'].notna()]
df_clean = df_clean[df_clean['brand'].notna()]

# Reset index
df_clean = df_clean.reset_index(drop=True)

# Step 7: Display summary
print(f"\nCleaned shape: {df_clean.shape}")
print(f"\nRemoved {len(df) - len(df_clean)} rows with missing critical data")
print("\nFirst few rows:")
print(df_clean.head())
print("\nData types:")
print(df_clean.dtypes)
print("\nMissing values:")
print(df_clean.isnull().sum())
print("\nBasic statistics:")
print(df_clean.describe())

# Save cleaned data
df_clean.to_csv('flipkart_cleaned.csv', index=False)
print("\n✓ Cleaned data saved to 'flipkart_cleaned.csv'")

# Additional insights
print("\n" + "="*50)
print("DATA SUMMARY")
print("="*50)
print(f"Total products: {len(df_clean)}")
print(f"Unique brands: {df_clean['brand'].nunique()}")
print(f"Top 5 brands:")
print(df_clean['brand'].value_counts().head())
print(f"\nPrice range: ₹{df_clean['current_price'].min():,.0f} - ₹{df_clean['current_price'].max():,.0f}")
print(f"Average price: ₹{df_clean['current_price'].mean():,.0f}")
print(f"Average discount: {df_clean['discount_percent'].mean():.1f}%")
print(f"Average rating: {df_clean['rating'].mean():.2f}")



Cleaned shape: (1080, 18)

Removed 0 rows with missing critical data

First few rows:
       brand                                       product_name  \
0    Thomson  Thomson Phoenix 2025 Edition 108 cm (43 inch) ...   
1    Thomson  Thomson Phoenix 2025 Edition 126 cm (50 inch) ...   
2     Foxsky  Foxsky 80 cm (32 inch) HD Ready LED Smart Andr...   
3  Blaupunkt  Blaupunkt Quantum Dot 100 cm (40 inch) QLED Fu...   
4       SONY  SONY BRAVIA 2 II 138.8 cm (55 inch) Ultra HD (...   

   screen_size_cm  screen_size_inch screen_resolution display_type  rating  \
0           108.0              43.0     Ultra HD (4K)         QLED     4.3   
1           126.0              50.0     Ultra HD (4K)         QLED     4.3   
2            80.0              32.0          HD Ready          LED     4.1   
3           100.0              40.0           Full HD         QLED     4.4   
4             8.0              55.0     Ultra HD (4K)          LED     4.7   

   num_ratings  num_reviews  current_pric

In [23]:
df_clean.head(100)

,brand,product_name,screen_size_cm,screen_size_inch,screen_resolution,display_type,rating,num_ratings,num_reviews,current_price,original_price,discount_percent,discount_amount,exchange_value,launch_year,sound_watts,model_id_clean,warranty_clean
0,Thomson,Thomson Phoenix 2025 Edition 108 cm (43 inch) ...,108.0,43.0,Ultra HD (4K),QLED,4.3,34911.0,5706.0,18889.0,28999.0,34.0,10110.0,5400.0,2025,50,43QAI1005,1 Year Warranty on Product
1,Thomson,Thomson Phoenix 2025 Edition 126 cm (50 inch) ...,126.0,50.0,Ultra HD (4K),QLED,4.3,34911.0,5706.0,23999.0,40999.0,41.0,17000.0,5400.0,2025,48,50QAI1015,1 Year Warranty on Product
2,Foxsky,Foxsky 80 cm (32 inch) HD Ready LED Smart Andr...,80.0,32.0,HD Ready,LED,4.1,12502.0,2521.0,6999.0,22499.0,68.0,15500.0,NaN,2023,30,32 FSELS PRO,1-Year Foxsky Standard Warranty on Product fro...
3,Blaupunkt,Blaupunkt Quantum Dot 100 cm (40 inch) QLED Fu...,100.0,40.0,Full HD,QLED,4.4,12512.0,2934.0,13799.0,19999.0,31.0,6200.0,5400.0,2025,48,40QD7070,1 Year Warranty on Product & 6 Months on Acces...
4,SONY,SONY BRAVIA 2 II 138.8 cm (55 inch) Ultra HD (...,8.0,55.0,Ultra HD (4K),LED,4.7,27797.0,3527.0,55990.0,91900.0,39.0,35910.0,5400.0,2024,20,K-55S25EM2,1 year warranty provided by the manufacturer f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LG,LG AI TV UA8200 126 cm (50 inch) Ultra HD (4K)...,126.0,50.0,Ultra HD (4K),LED,4.4,31838.0,2178.0,35990.0,64490.0,44.0,28500.0,NaN,2025,20,50UA82006LA,Warranty Information: 1-Year LG India Standard...
96,TCL,TCL T6C 139 cm (55 inch) QLED Ultra HD (4K) Sm...,139.0,55.0,Ultra HD (4K),QLED,4.0,2487.0,175.0,32990.0,93990.0,64.0,61000.0,5400.0,2025,24,55T6C,2 Years Product Warranty
97,Samsung,Samsung Crystal 4K Infinity Vision 125 cm (50 ...,125.0,50.0,Ultra HD (4K),LED,4.3,64260.0,4359.0,36990.0,52400.0,29.0,15410.0,5400.0,2025,20,UA50UE84AFULXL,2 Year Warranty (1 Year Standard Warranty + 1 ...
98,TCL,TCL C6K 139 cm (55 inch) Ultra HD (4K) Mini LE...,139.0,55.0,Ultra HD (4K),Mini LED,4.1,585.0,46.0,45990.0,119990.0,61.0,74000.0,7900.0,2025,40,55C6K,2 Years Product Warranty


In [24]:
# The df_clean DataFrame is already available from the previous execution

import numpy as np

# --- 1. Handle 'warranty_clean' column: Convert to numerical categories ---

print("--- Converting 'warranty_clean' to numerical categories ---")

# Convert the column to numeric categories.
# pd.factorize assigns a unique integer to each unique text value.
# NaN values will be assigned -1 by factorize, which will be retained as the 'Missing' category.
df_clean['warranty_category'], unique_warranties = pd.factorize(df_clean['warranty_clean'])

# Display the mapping (Category ID -> Original Text)
# The index 0 corresponds to the first unique warranty text, 1 to the second, and so on.
warranty_map = pd.Series(unique_warranties, index=np.arange(len(unique_warranties)))
print("\nMapping of Warranty Categories:")
print(warranty_map)

# Drop the original 'warranty_clean' column as requested by the conversion
df_clean = df_clean.drop(columns=['warranty_clean'])

# --- 2. Fill all null values ---

print("\n--- Filling Null Values ---")

# 1. Fill Numerical Missing values:
# Columns for median imputation (more robust to outliers than mean)
median_cols = [
    'screen_size_cm', 'screen_size_inch', 'rating', 'num_ratings', 'num_reviews',
    'original_price', 'discount_percent', 'discount_amount'
]
for col in median_cols:
    df_clean[col].fillna(df_clean[col].median(), inplace=True)

# Column for zero imputation (missing value means no exchange value)
df_clean['exchange_value'].fillna(0, inplace=True)

print("\nMissing values after imputation:")
print(df_clean.isnull().sum())

print("\nFirst five rows of the final cleaned DataFrame:")
print(df_clean.head())


--- Converting 'warranty_clean' to numerical categories ---

Mapping of Warranty Categories:
0                             1 Year Warranty on Product
1      1-Year Foxsky Standard Warranty on Product fro...
2      1 Year Warranty on Product & 6 Months on Acces...
3      1 year warranty provided by the manufacturer f...
4                               1 Year Domestic Warranty
                             ...                        
180    1 Year LG Comprehensive Warranty and Additiona...
181                                 1 year comprehensive
182    1 Year Redmi Comprehensive Warranty and Additi...
183                                       1Year Warranty
184                             1 YEAR FREE HOME SERVICE
Length: 185, dtype: object

--- Filling Null Values ---

Missing values after imputation:
brand                0
product_name         0
screen_size_cm       0
screen_size_inch     0
screen_resolution    0
display_type         0
rating               0
num_ratings          0
num_re

/var/folders/c3/1_4ygmt13z74mbwl3gvsd8600000gn/T/ipykernel_5157/3032259335.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean[col].fillna(df_clean[col].median(), inplace=True)
/var/folders/c3/1_4ygmt13z74mbwl3gvsd8600000gn/T/ipykernel_5157/3032259335.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting v

In [25]:
import pandas as pd
import re
import numpy as np

# --- CORRECTION APPLIED HERE ---
# Use the 'python' engine to handle non-uniform number of columns in the CSV.
df = pd.read_csv('flipkart (1).csv', engine='python')
# ------------------------------

# Display original column names
print("Original columns:", df.columns.tolist())
print(f"Original shape: {df.shape}")

# Step 1: Assign proper column names based on the data structure
column_names = [
    'product_name',
    'rating',
    'num_ratings',
    'and_symbol_1',
    'num_reviews',
    'screen_type',
    'model_id',
    'launch_year',
    'sound_output',
    'warranty',
    'current_price',
    'original_price',
    'discount_percent',
    'exchange_label',
    'exchange_value'
]

df.columns = column_names

# Step 2: Remove completely empty rows
df = df.dropna(how='all')

# Step 3: Remove rows where product_name is empty or very short
df = df[df['product_name'].notna()]
df = df[df['product_name'].str.len() > 10]

# --- Cleaning functions (unchanged) ---
def clean_price(price_str):
    """Extract numeric price from string like '₹18,889'"""
    if pd.isna(price_str):
        return np.nan
    # Remove currency symbols and commas
    cleaned = re.sub(r'[₹,]', '', str(price_str))
    try:
        return float(cleaned)
    except:
        return np.nan

def clean_rating(rating_str):
    """Extract rating number"""
    if pd.isna(rating_str):
        return np.nan
    try:
        return float(rating_str)
    except:
        return np.nan

def clean_number(num_str):
    """Extract numbers from strings like '34,911 Ratings'"""
    if pd.isna(num_str):
        return np.nan
    # Extract numbers and remove commas
    match = re.search(r'[\d,]+', str(num_str))
    if match:
        cleaned = re.sub(r',', '', match.group())
        try:
            return int(cleaned)
        except:
            return np.nan
    return np.nan

def clean_discount(discount_str):
    """Extract discount percentage"""
    if pd.isna(discount_str):
        return np.nan
    # Extract number before '%'
    match = re.search(r'(\d+)%', str(discount_str))
    if match:
        return int(match.group(1))
    return np.nan

def extract_screen_info(screen_str):
    """Extract screen resolution and type"""
    if pd.isna(screen_str):
        return np.nan, np.nan
    parts = str(screen_str).split('|')
    resolution = parts[0].strip() if len(parts) > 0 else np.nan
    display_type = parts[1].strip() if len(parts) > 1 else np.nan
    return resolution, display_type

def extract_year(year_str):
    """Extract year from 'Launch Year: 2025'"""
    if pd.isna(year_str):
        return np.nan
    match = re.search(r'(\d{4})', str(year_str))
    if match:
        return int(match.group(1))
    return np.nan

def extract_watts(sound_str):
    """Extract watts from 'Total Sound Output: 50 W'"""
    if pd.isna(sound_str):
        return np.nan
    match = re.search(r'(\d+)\s*W', str(sound_str))
    if match:
        return int(match.group(1))
    return np.nan

def extract_model_id(model_str):
    """Clean model ID"""
    if pd.isna(model_str):
        return np.nan
    # Extract everything after 'Model ID:'
    match = re.search(r'Model ID:\s*(.+)', str(model_str))
    if match:
        return match.group(1).strip()
    return str(model_str).strip()

def clean_warranty(warranty_str):
    """Clean warranty text"""
    if pd.isna(warranty_str):
        return np.nan
    # Remove extra spaces and special characters
    cleaned = re.sub(r'\s+', ' ', str(warranty_str))
    return cleaned.strip()

# Apply cleaning functions
print("\nCleaning data...")

df['rating'] = df['rating'].apply(clean_rating)
df['num_ratings'] = df['num_ratings'].apply(clean_number)
df['num_reviews'] = df['num_reviews'].apply(clean_number)
df['current_price'] = df['current_price'].apply(clean_price)
df['original_price'] = df['original_price'].apply(clean_price)
df['discount_percent'] = df['discount_percent'].apply(clean_discount)
df['exchange_value'] = df['exchange_value'].apply(clean_price)

# Extract screen information
df[['screen_resolution', 'display_type']] = df['screen_type'].apply(
    lambda x: pd.Series(extract_screen_info(x))
)

df['launch_year'] = df['launch_year'].apply(extract_year)
df['sound_watts'] = df['sound_output'].apply(extract_watts)
df['model_id_clean'] = df['model_id'].apply(extract_model_id)
df['warranty_clean'] = df['warranty'].apply(clean_warranty)

# Extract brand from product name
def extract_brand(product_name):
    """Extract brand name (first word typically)"""
    if pd.isna(product_name):
        return np.nan
    words = str(product_name).split()
    return words[0] if words else np.nan

df['brand'] = df['product_name'].apply(extract_brand)

# Extract screen size from product name
def extract_screen_size(product_name):
    """Extract screen size in cm and inches"""
    if pd.isna(product_name):
        return np.nan, np.nan
    # Pattern: "108 cm (43 inch)"
    pattern = r'(\d+)\s*cm\s*\((\d+)\s*inch\)'
    match = re.search(pattern, str(product_name))
    if match:
        return int(match.group(1)), int(match.group(2))
    return np.nan, np.nan

df[['screen_size_cm', 'screen_size_inch']] = df['product_name'].apply(
    lambda x: pd.Series(extract_screen_size(x))
)

# Calculate discount amount
df['discount_amount'] = df['original_price'] - df['current_price']
final_columns = [
    'brand',
    'product_name',
    'screen_size_cm',
    'screen_size_inch',
    'screen_resolution',
    'display_type',
    'rating',
    'num_ratings',
    'num_reviews',
    'current_price',
    'original_price',
    'discount_percent',
    'discount_amount',
    'exchange_value',
    'launch_year',
    'sound_watts',
    'model_id_clean',
    'warranty_clean'
]

df_clean = df[final_columns].copy()

# Step 6: Remove rows with critical missing data
df_clean = df_clean[df_clean['current_price'].notna()]
df_clean = df_clean[df_clean['brand'].notna()]

# Reset index
df_clean = df_clean.reset_index(drop=True)

# Step 7: Display summary
print(f"\nCleaned shape: {df_clean.shape}")
print(f"\nRemoved {len(df) - len(df_clean)} rows with missing critical data")
print("\nFirst few rows:")
print(df_clean.head())
print("\nData types:")
print(df_clean.dtypes)
print("\nMissing values:")
print(df_clean.isnull().sum())
print("\nBasic statistics:")
print(df_clean.describe())

# Save cleaned data
df_clean.to_csv('flipkart_cleaned.csv', index=False)
print("\n✓ Cleaned data saved to 'flipkart_cleaned.csv'")

Original columns: ['KzDlHZ', 'XQDdHH', 'Wphh3N', 'hG7V+4', 'Wphh3N 2', 'J+igdf', 'J+igdf 2', 'J+igdf 3', 'J+igdf 4', 'J+igdf 5', 'Nx9bqj', 'yRaY8j', 'UkUFwK', 'yiggsN 2', 'yiggsN 3']
Original shape: (1126, 15)

Cleaning data...

Cleaned shape: (1080, 18)

Removed 0 rows with missing critical data

First few rows:
       brand                                       product_name  \
0    Thomson  Thomson Phoenix 2025 Edition 108 cm (43 inch) ...   
1    Thomson  Thomson Phoenix 2025 Edition 126 cm (50 inch) ...   
2     Foxsky  Foxsky 80 cm (32 inch) HD Ready LED Smart Andr...   
3  Blaupunkt  Blaupunkt Quantum Dot 100 cm (40 inch) QLED Fu...   
4       SONY  SONY BRAVIA 2 II 138.8 cm (55 inch) Ultra HD (...   

   screen_size_cm  screen_size_inch screen_resolution display_type  rating  \
0           108.0              43.0     Ultra HD (4K)         QLED     4.3   
1           126.0              50.0     Ultra HD (4K)         QLED     4.3   
2            80.0              32.0          HD 

In [26]:
# Assuming 'df_clean' is the cleaned DataFrame
df_final = df_clean.drop(columns=['exchange_value', 'warranty_clean'])


df_final.head()


,brand,product_name,screen_size_cm,screen_size_inch,screen_resolution,display_type,rating,num_ratings,num_reviews,current_price,original_price,discount_percent,discount_amount,launch_year,sound_watts,model_id_clean
0,Thomson,Thomson Phoenix 2025 Edition 108 cm (43 inch) ...,108.0,43.0,Ultra HD (4K),QLED,4.3,34911.0,5706.0,18889.0,28999.0,34.0,10110.0,2025,50,43QAI1005
1,Thomson,Thomson Phoenix 2025 Edition 126 cm (50 inch) ...,126.0,50.0,Ultra HD (4K),QLED,4.3,34911.0,5706.0,23999.0,40999.0,41.0,17000.0,2025,48,50QAI1015
2,Foxsky,Foxsky 80 cm (32 inch) HD Ready LED Smart Andr...,80.0,32.0,HD Ready,LED,4.1,12502.0,2521.0,6999.0,22499.0,68.0,15500.0,2023,30,32 FSELS PRO
3,Blaupunkt,Blaupunkt Quantum Dot 100 cm (40 inch) QLED Fu...,100.0,40.0,Full HD,QLED,4.4,12512.0,2934.0,13799.0,19999.0,31.0,6200.0,2025,48,40QD7070
4,SONY,SONY BRAVIA 2 II 138.8 cm (55 inch) Ultra HD (...,8.0,55.0,Ultra HD (4K),LED,4.7,27797.0,3527.0,55990.0,91900.0,39.0,35910.0,2024,20,K-55S25EM2


In [28]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brand              1080 non-null   object 
 1   product_name       1080 non-null   object 
 2   screen_size_cm     1071 non-null   float64
 3   screen_size_inch   1071 non-null   float64
 4   screen_resolution  1080 non-null   object 
 5   display_type       1080 non-null   object 
 6   rating             846 non-null    float64
 7   num_ratings        846 non-null    float64
 8   num_reviews        846 non-null    float64
 9   current_price      1080 non-null   float64
 10  original_price     1070 non-null   float64
 11  discount_percent   1070 non-null   float64
 12  discount_amount    1070 non-null   float64
 13  launch_year        1080 non-null   int64  
 14  sound_watts        1080 non-null   int64  
 15  model_id_clean     1080 non-null   object 
dtypes: float64(9), int64(2),

In [29]:
df_final.isnull().sum()

brand                  0
product_name           0
screen_size_cm         9
screen_size_inch       9
screen_resolution      0
display_type           0
rating               234
num_ratings          234
num_reviews          234
current_price          0
original_price        10
discount_percent      10
discount_amount       10
launch_year            0
sound_watts            0
model_id_clean         0
dtype: int64

In [30]:
# Assuming 'df_final' is your DataFrame after dropping 'exchange_value' and 'warranty_clean'

# 1. Fill Numerical Columns (with Median for robustness against outliers)
# Columns like rating, num_ratings, prices, sound_watts, screen_size_inch, etc.
numerical_cols = [
    'rating', 'num_ratings', 'num_reviews', 'current_price', 'original_price',
    'discount_percent', 'discount_amount', 'screen_size_cm', 'screen_size_inch',
    'launch_year', 'sound_watts'
]

for col in numerical_cols:
    median_val = df_final[col].median()
    df_final[col] = df_final[col].fillna(median_val)

# 2. Fill Categorical/Text Columns (with 'Unknown')
# Columns like screen_resolution, display_type, model_id_clean
categorical_cols = [
    'screen_resolution', 'display_type', 'model_id_clean'
]

for col in categorical_cols:
    df_final[col] = df_final[col].fillna('Unknown')

# 3. Verify changes
print("\nMissing values after imputation:")
print(df_final.isnull().sum())


Missing values after imputation:
brand                0
product_name         0
screen_size_cm       0
screen_size_inch     0
screen_resolution    0
display_type         0
rating               0
num_ratings          0
num_reviews          0
current_price        0
original_price       0
discount_percent     0
discount_amount      0
launch_year          0
sound_watts          0
model_id_clean       0
dtype: int64


In [31]:
# Make sure you run this line after cleaning and dropping columns!
df_final.to_csv('flipkart_final.csv', index=False)

In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

# --- Load Data (Run this first, assuming 'flipkart_final.csv' is in your environment) ---
df = pd.read_csv('flipkart_final.csv')

In [33]:
 import pandas as pd

# 1. Load the dataset
df = pd.read_csv('flipkart_final.csv')

# 2. Remove Duplicate Rows
# This keeps the first occurrence of each row and removes the 124 duplicates.
df_cleaned = df.drop_duplicates()

print(f"Rows before cleaning: {len(df)}")
print(f"Rows after removing duplicates: {len(df_cleaned)}")

# 3. Fix the 'screen_size_cm' column
# We overwrite the bad values by recalculating them from the correct 'screen_size_inch' column.
# Formula: 1 inch = 2.54 cm
df_cleaned['screen_size_cm'] = df_cleaned['screen_size_inch'] * 2.54

# Round to 1 decimal place to keep it tidy
df_cleaned['screen_size_cm'] = df_cleaned['screen_size_cm'].round(1)

# 4. Save the cleaned data to a new CSV file
df_cleaned.to_csv('flipkart_cleaned.csv', index=False)

print("Success! Data cleaned and saved to 'flipkart_cleaned.csv'.")

# Verification: Show a few rows to prove the fix
print(df_cleaned[['brand', 'screen_size_inch', 'screen_size_cm']].head())

Rows before cleaning: 1080
Rows after removing duplicates: 956
Success! Data cleaned and saved to 'flipkart_cleaned.csv'.
       brand  screen_size_inch  screen_size_cm
0    Thomson              43.0           109.2
1    Thomson              50.0           127.0
2     Foxsky              32.0            81.3
3  Blaupunkt              40.0           101.6
4       SONY              55.0           139.7


/var/folders/c3/1_4ygmt13z74mbwl3gvsd8600000gn/T/ipykernel_5157/2902525927.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['screen_size_cm'] = df_cleaned['screen_size_inch'] * 2.54
/var/folders/c3/1_4ygmt13z74mbwl3gvsd8600000gn/T/ipykernel_5157/2902525927.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['screen_size_cm'] = df_cleaned['screen_size_cm'].round(1)


In [34]:
df_cleaned.head()

,brand,product_name,screen_size_cm,screen_size_inch,screen_resolution,display_type,rating,num_ratings,num_reviews,current_price,original_price,discount_percent,discount_amount,launch_year,sound_watts,model_id_clean
0,Thomson,Thomson Phoenix 2025 Edition 108 cm (43 inch) ...,109.2,43.0,Ultra HD (4K),QLED,4.3,34911.0,5706.0,18889.0,28999.0,34.0,10110.0,2025,50,43QAI1005
1,Thomson,Thomson Phoenix 2025 Edition 126 cm (50 inch) ...,127.0,50.0,Ultra HD (4K),QLED,4.3,34911.0,5706.0,23999.0,40999.0,41.0,17000.0,2025,48,50QAI1015
2,Foxsky,Foxsky 80 cm (32 inch) HD Ready LED Smart Andr...,81.3,32.0,HD Ready,LED,4.1,12502.0,2521.0,6999.0,22499.0,68.0,15500.0,2023,30,32 FSELS PRO
3,Blaupunkt,Blaupunkt Quantum Dot 100 cm (40 inch) QLED Fu...,101.6,40.0,Full HD,QLED,4.4,12512.0,2934.0,13799.0,19999.0,31.0,6200.0,2025,48,40QD7070
4,SONY,SONY BRAVIA 2 II 138.8 cm (55 inch) Ultra HD (...,139.7,55.0,Ultra HD (4K),LED,4.7,27797.0,3527.0,55990.0,91900.0,39.0,35910.0,2024,20,K-55S25EM2


In [35]:
df_cleaned.to_csv('flipkart_cleaned.csv', index=False)